## Colormaps

เมื่อคุณต้องการแยกความแตกต่างระหว่างพื้นที่ต่าง ๆ  
โดยไม่ต้องการสื่อถึงความสัมพันธ์เชิงลำดับหรือปริมาณใด ๆ  
การใช้ชุดสีแบบเชิงคุณภาพ (qualitative colormap) เป็นตัวเลือกที่เหมาะสมที่สุด

ในแบบฝึกหัดนี้ คุณจะเปรียบเทียบการใช้ชุดสีแบบเชิงคุณภาพกับชุดสีแบบลำดับค่า (sequential หรือ quantitative colormap) โดยใช้ GeoDataFrame ของเขตการศึกษา (`school_districts`)

In [ ]:
import geopandas as gpd

school_districts = gpd.read_file('../../data/raw/school_districts.geojson')

school_districts.head()

In [ ]:
import matplotlib.pyplot as plt
import contextily as ctx
# Set legend style
lgnd_kwds = {'title': 'School Districts',
               'loc': 'upper left', 'bbox_to_anchor': (1, 1.03), 'ncol': 1}

# Plot the school districts using the tab20 colormap (qualitative)
school_districts.plot(column = 'district', cmap = 'tab20', legend = True, legend_kwds  = lgnd_kwds, alpha = 0.3, figsize=(10, 10))
ctx.add_basemap(plt.gca(), source=ctx.providers.CartoDB.Positron)
plt.xlabel('Latitude')
plt.ylabel('Longitude')
plt.title('Nashville School Districts')
plt.show();

In [ ]:
# Set legend style
lgnd_kwds = {'title': 'School Districts',
               'loc': 'upper left', 'bbox_to_anchor': (1, 1.03), 'ncol': 1}

# Plot the school districts using the summer colormap (sequential)
school_districts.plot(column = 'district', cmap = 'summer', legend = True, legend_kwds = lgnd_kwds, alpha = 0.3, figsize=(10, 10))
ctx.add_basemap(plt.gca(), source=ctx.providers.CartoDB.Positron)
plt.xlabel('Latitude')
plt.ylabel('Longitude')
plt.title('Nashville School Districts')
plt.show();

In [ ]:
# Set legend style
lgnd_kwds = {'title': 'School Districts',
               'loc': 'upper left', 'bbox_to_anchor': (1, 1.03), 'ncol': 1}

# Plot the school districts using Set3 colormap without the column argument
school_districts.plot(column = 'district', cmap = 'Set3', legend = True, legend_kwds  = lgnd_kwds, alpha = 0.3, figsize=(10, 10))
ctx.add_basemap(plt.gca(), source=ctx.providers.CartoDB.Positron)
plt.xlabel('Latitude')
plt.ylabel('Longitude')
plt.title('Nashville School Districts')
plt.show();

## แผนที่ย่านต่าง ๆ ในแนชวิลล์ (Nashville)

ในครั้งนี้ คุณจะอ่านไฟล์ GeoJSON เข้าสู่ GeoDataFrame  
เพื่อสำรวจตำแหน่งของย่านต่าง ๆ ในเมืองแนชวิลล์แบบคร่าว ๆ

In [ ]:
# Read in the neighborhoods geojson file
neighborhoods = gpd.read_file('../../data/raw/neighborhoods.geojson')

# Print the first few rows of neighborhoods
neighborhoods.head()


In [ ]:
# Plot the neighborhoods, color according to name and use the Dark2 colormap
neighborhoods.plot(column = 'name', cmap = 'Dark2')
plt.xlabel('Latitude')
plt.ylabel('Longitude')
plt.title('Nashville Neighborhoods')
# Show the plot.
plt.show()

In [ ]:
# แปลงเป็น Web Mercator
neigh_3857 = neighborhoods.to_crs(epsg=3857)

ax = neigh_3857.plot(column="name", cmap="Dark2", figsize=(10, 10), alpha=0.7)

ctx.add_basemap(ax, source=ctx.providers.CartoDB.Positron, crs=neigh_3857.crs)

ax.set_axis_off()
ax.set_title("Nashville Neighborhoods")
plt.show()

## Changing coordinate reference systems


In [ ]:
# Check the coordinate reference system (CRS) of the school districts 
print(school_districts.crs)

In [ ]:
# Convert the crs to epsg:3857
school_districts = school_districts.to_crs(epsg=3857)
# Check the coordinate reference system (CRS) of the school districts again to confirm it has been converted
print(school_districts.crs)

## Construct a GeoDataFrame from a DataFrame

In [ ]:
import pandas as pd
from shapely.geometry import Point

art = pd.read_csv('../../data/raw/public_art.csv')
art.head()

In [ ]:
# Create a geometry column from the longitude and latitude columns
art['geometry'] = art.apply(lambda x: Point(float(x['Longitude']), float(x['Latitude'])), axis=1)

# Create a GeoDataFrame from the art DataFrame, using the geometry column and setting the CRS to neighborhoods CRS
art_gdf = gpd.GeoDataFrame(art, geometry='geometry', crs=neighborhoods.crs)
art_gdf.head()

In [ ]:
# Check the coordinate reference system (CRS) of the art_gdf GeoDataFrame
print(art_gdf.crs)

## Spatial join practice

In [ ]:
# Spatially join art_gdf to neighborhoods, using the 'intersects' predicate 
art_intersect_neighborhoods = gpd.sjoin(art_gdf, neighborhoods, predicate='intersects')
print(art_intersect_neighborhoods.shape)

In [ ]:
# Spatially join art_gdf to neighborhoods, using the 'within' predicate 
art_within_neighborhoods = gpd.sjoin(art_gdf, neighborhoods, predicate='within')
print(art_within_neighborhoods.shape)


In [ ]:
# Spatially join art_gdf to neighborhoods, using the 'contains' predicate 
art_contains_neighborhoods = gpd.sjoin(art_gdf, neighborhoods, predicate='contains')
print(art_contains_neighborhoods.shape)


## การค้นหาย่านที่มีงานศิลปะสาธารณะมากที่สุด

ในขั้นตอนนี้ คุณจะวิเคราะห์ข้อมูลเพื่อค้นหาว่า  
ย่านใดในเมืองแนชวิลล์มีจำนวนผลงานศิลปะสาธารณะมากที่สุด

In [ ]:
# Spatially join neighborhoods with art_gdf
neighborhoods_with_art = gpd.sjoin(art_gdf, neighborhoods, predicate='within')
neighborhoods_with_art.head()


## การรวมข้อมูลจุดภายในพื้นที่หลายเหลี่ยม (Aggregating Points Within Polygons)

เมื่อคุณได้ทำการเชื่อมข้อมูลเชิงพื้นที่ (spatial join) ระหว่างผลงานศิลปะและย่านต่าง ๆ แล้ว  
คุณสามารถจัดกลุ่ม (group), รวมค่า (aggregate) และเรียงลำดับ (sort) ข้อมูล  
เพื่อค้นหาว่าย่านใดมีงานศิลปะสาธารณะมากที่สุด

คุณสามารถนับจำนวนชื่อผลงานศิลปะ (artwork titles)  
เพื่อดูว่าแต่ละย่านมีผลงานศิลปะจำนวนกี่ชิ้น

In [ ]:
# Get name and title from neighborhood_art and group by name
neighborhood_art_grouped = neighborhoods_with_art[['name', 'Title']].groupby('name')

# Aggregate the grouped data and count the artworks within each polygon
neighborhood_art_grouped.agg('count').sort_values(by = 'Title', ascending = False)

## การพล็อตย่าน Urban Residents และผลงานศิลปะ

ตอนนี้คุณทราบแล้วว่างานศิลปะส่วนใหญ่อยู่ในย่าน **Urban Residents**

ในแบบฝึกหัดนี้ คุณจะสร้างแผนที่แสดงผลงานศิลปะภายในย่านดังกล่าว โดยมีขั้นตอนดังนี้:

1. เลือกเฉพาะข้อมูลผลงานศิลปะในย่าน Urban Residents จาก `neighborhood_art` แล้วเก็บไว้ในตัวแปร `urban_art`
2. เลือกเฉพาะ polygon ของย่าน Urban Residents จาก `neighborhoods` แล้วเก็บไว้ในตัวแปร `urban_polygon`
3. พล็อต polygon ก่อน และกำหนดให้เป็นแกน `ax`
4. จากนั้นเพิ่มการพล็อตจุดผลงานศิลปะลงบนแผนที่เดียวกัน

In [ ]:
# Create urban_art from neighborhood_art where the neighborhood name is Urban Residents
urban_art = neighborhoods_with_art.loc[neighborhoods_with_art['name'] == 'Urban Residents']

# Get just the Urban Residents neighborhood polygon and save it as urban_polygon
urban_polygon = neighborhoods.loc[neighborhoods['name'] == "Urban Residents"]

# Plot the urban_polygon as ax 
ax = urban_polygon.plot(color = 'lightgreen')

# Add a plot of the urban_art and show it
urban_art.plot( ax = ax, column = 'Type', legend = True)
plt.show()

In [ ]:
# แปลงเป็น Web Mercator
urban_art_3857 = urban_art.to_crs(epsg=3857)
urban_polygon_3857 = urban_polygon.to_crs(epsg=3857)

ax = urban_art_3857.plot(column="Type", cmap="Dark2", figsize=(10, 10), alpha=0.7, legend = True)
urban_polygon_3857.plot(ax=ax, color='lightgreen', alpha=0.2)
ctx.add_basemap(ax, source=ctx.providers.CartoDB.Positron, crs=urban_art_3857.crs)

ax.set_axis_off()
plt.show()